Here, I am doing semnatic analysis for facebook comments on different posts. I am using a classic machine learning approach. 

I wont put data public here, but if you have your own facebook page (you are the admin of a page) you can download the comments from the posts and run this code for your own posts. I am randomly taking comments from different posts and do the training on them. The results are not super good yet as I need much higher numbrs of labels. 


Here, I explain how we make words into vectors (text vectorization). One method to use is called CountVectorizer and you can have it imported from python: 
from sklearn.feature_extraction.text import CountVectorizer
Tokenization and filtering stopworks are all included in this library (so no need to do use many parts of previous cell!). This method counts the number of times a word appears in the document and make a sparse matrix relatively. This is exactly what bag of word (BOW) will do for us!  
I use the example from https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html python documentary:

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

1) For the corpus awe distinguish all the used words which are: 
['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']

2) For each sentence we see if the word is appeared, in this case for "This is the first document" we will have:
[0, 1, 1, 1, 0, 0, 1, 0, 1] 
This means that in the first sentence "and" has not been used. We can go ahead and make a matrix represnting the corpus. The commands can be found in the above documnet.

Remeber, in BOW we loose the word order and the counts are not normalized. However, we can count token paris (n-grams) which CountVectorizer in Python is capable of doing it. 

Another usefull thing we use is called TF/IDF:

I explain TF first: Term Frequency tf(t,d) is the frequency for term (or n-grams) in a documemnt. We can calculate it as follows:

\begin{equation*}
tf(t,d) = \frac {f_{t,d}}{\sum_{t' \in d} f_{t',d}}
\end{equation*}

where $f_{t,d}$ is the raw counts of the term and $\sum_{t'} f_{t',d}$ is the summation of all terms; so the TF gives a probability of a term being used. We can also have the log norm of the term frquency:

\begin{equation*}
tf(t,d) =  1 + \log(f_{t,d})
\end{equation*}

Now I explain Inverese Document Frequency (IDF):

We define $N = D$ as total number of documents in a corpus and $\mid d \in D: t \in d \mid$ as number of documents when terms t appears. And, we can write:
\begin{equation*}
idf(t,D) =  \log \frac{N}{\mid d \in D: t \in d \mid}
\end{equation*}

So, TF-IDF can be written as:

\begin{equation*}
tfidf(t,d,D) =  tf(t,d).idf(t, D)
\end{equation*}

Now, this is obvious that why a high weight TF-IDF is desirable: because it means that it has high term frequency in a document, but low frequency in the WHOLE collection of documnets in the corpus. 
To make TFIDF for tokens (n_grams) in our corpus we can use the following library:

from sklearn.feature_extraction.text import TfidfTransformer

So, when we have our text we need to use both vectorizer and tfidf transformer. And, then our data is ready to be trained. In order to make 1) vectorizer 2) transformer 3) classifier we can use Pipeline calss in scikit-learn.



In [2]:
import pandas as pd

The next few cells are about loading my comments which I cannot share with you :( 

In [36]:
import pandas as pd
March03 = pd.read_csv("labels_March03.csv")
March12 = pd.read_csv("labels_March12.csv")
March13 = pd.read_csv("labels_March13.csv")
March16 = pd.read_csv("labels_March16.csv")
March27 = pd.read_csv("labels_March27_2nd.csv")
April9 = pd.read_csv("labels_April9.csv")
April13 = pd.read_csv("labels_April13.csv")

In [4]:
Labels = pd.concat([March03, March12, March13, March16, March27, April9, April13])

I only look at the comments with labels with high confidene in them 

In [8]:
Labels = Labels[["comment","label", "conffidence"]]
Labels = Labels[Labels.conffidence>.7]
Labels = Labels.reset_index(drop = True)

In [9]:
Labels.count()

comment        4348
label          4348
conffidence    4348
dtype: int64

In [10]:
Labels = Labels[Labels.label != "I cannot tell"]

In [11]:
Labels.count()

comment        3842
label          3842
conffidence    3842
dtype: int64

In [12]:
Labels['label'] = Labels['label'].replace("disgusted", "angry")
Labels['label'] = Labels['label'].replace("angry/disgusted", "angry")
Labels['label'] = Labels['label'].replace("worried/scared", "worried")
Labels['label'] = Labels['label'].replace("sad", "worried")
Labels['label'] = Labels['label'].replace("scared", "worried")
Labels['label'] = Labels['label'].replace("indifferent", "neutral")
Labels['label'] = Labels['label'].replace("happy", "positive")
Labels['label'] = Labels['label'].replace("supportive", "positive")
Labels['label'] = Labels['label'].replace("happy/supportive/positive", "positive")
Labels['label'] = Labels['label'].replace("worried/scared/sad", "worried")



In [13]:
# Labels['label'] = Labels['label'].replace("angry", "negative")
# Labels['label'] = Labels['label'].replace("worried", "negative")

In [14]:
neutral_count = Labels[Labels.label == 'neutral'].count()
positive_count = Labels[Labels.label == 'positive'].count()
worried_count = Labels[Labels.label == 'worried'].count()
angry_count = Labels[Labels.label == 'angry'].count()
#positive_count = Labels[Labels.label == 'positive'].count()
#negative_count = Labels[Labels.label == 'negative'].count()

In [15]:
{'neutral_count': neutral_count, 'positive_count': positive_count, 'worried_count':worried_count, 'angry_count': angry_count , 'positive_count':positive_count }

{'angry_count': comment        1772
 label          1772
 conffidence    1772
 dtype: int64, 'neutral_count': comment        856
 label          856
 conffidence    856
 dtype: int64, 'positive_count': comment        796
 label          796
 conffidence    796
 dtype: int64, 'worried_count': comment        417
 label          417
 conffidence    417
 dtype: int64}

In [37]:
# many people are angry! so, I am down sampling the angry comments!
angry = Labels[Labels.label == "angry"].sample(n=800, frac=None)
no_angry = Labels[Labels.label != "angry"]
Labels = pd.concat([angry, no_angry], axis = 0)
Labels = Labels.sample(frac=1).reset_index(drop=True)
Labels.count()

comment        2870
label          2870
conffidence    2870
dtype: int64

In [17]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

In [30]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
Labels['comment']=Labels.iloc[:,0].apply(clean_text)


In [19]:
Labels_train = Labels[0:2300]
Labels_test = Labels[2300:]


In [24]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(Labels_train['comment'],  Labels_train['label'])


from sklearn.metrics import classification_report
y_pred = nb.predict(Labels_test['comment'])


In [25]:
y_true = Labels_test['label']

In [26]:
y_true = y_true.to_numpy()

In [27]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

       angry       0.39      0.66      0.49       152
     neutral       0.38      0.36      0.37       173
    positive       0.44      0.40      0.42       165
     worried       1.00      0.03      0.05        80

    accuracy                           0.41       570
   macro avg       0.55      0.36      0.33       570
weighted avg       0.49      0.41      0.37       570



In [28]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(Labels_train['comment'],  Labels_train['label'])


y_pred = logreg.predict(Labels_test['comment'])

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

       angry       0.44      0.44      0.44       152
     neutral       0.42      0.37      0.40       173
    positive       0.45      0.42      0.43       165
     worried       0.25      0.36      0.30        80

    accuracy                           0.40       570
   macro avg       0.39      0.40      0.39       570
weighted avg       0.41      0.40      0.41       570



/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [29]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(Labels_train['comment'],  Labels_train['label'])


y_pred = sgd.predict(Labels_test['comment'])

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

       angry       0.43      0.48      0.46       152
     neutral       0.40      0.43      0.41       173
    positive       0.46      0.45      0.45       165
     worried       0.33      0.23      0.27        80

    accuracy                           0.42       570
   macro avg       0.40      0.40      0.40       570
weighted avg       0.42      0.42      0.42       570



In [30]:
pred = pd.DataFrame(y_pred )


In [31]:
pred.columns = ["prediction"]

In [32]:
Labels_test = Labels_test.reset_index(drop=True)

In [33]:
result = pd.concat([Labels_test, pred], axis = 1)